# Data Preparation

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
train_df = pd.read_csv('files/training_set_VU_DM.csv')
test_df = pd.read_csv('files/test_set_VU_DM.csv')

In [12]:
train_df["visitor_hist_starrating"].fillna(0, inplace=True)
train_df["visitor_hist_adr_usd"].fillna(-1, inplace=True)

# remove columns with more than 50% missing values
td = train_df.dropna(thresh=train_df.shape[0]*0.5,axis=1)
td = td.drop(['orig_destination_distance'], axis=1)
td

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool
0,1,2013-04-04 08:32:15,12,187,0.0,-1.0,219,893,3,3.5,...,23246,1,0,4,0,1,1,1,0,0
1,1,2013-04-04 08:32:15,12,187,0.0,-1.0,219,10404,4,4.0,...,23246,1,0,4,0,1,1,1,0,0
2,1,2013-04-04 08:32:15,12,187,0.0,-1.0,219,21315,3,4.5,...,23246,1,0,4,0,1,1,1,0,0
3,1,2013-04-04 08:32:15,12,187,0.0,-1.0,219,27348,2,4.0,...,23246,1,0,4,0,1,1,1,0,0
4,1,2013-04-04 08:32:15,12,187,0.0,-1.0,219,29604,4,3.5,...,23246,1,0,4,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,2013-06-30 19:55:18,5,219,0.0,-1.0,219,77700,3,4.0,...,16974,1,21,3,0,1,0,0,0,0
4958343,332785,2013-06-30 19:55:18,5,219,0.0,-1.0,219,88083,3,4.0,...,16974,1,21,3,0,1,0,0,0,0
4958344,332785,2013-06-30 19:55:18,5,219,0.0,-1.0,219,94508,3,3.5,...,16974,1,21,3,0,1,0,0,0,0
4958345,332785,2013-06-30 19:55:18,5,219,0.0,-1.0,219,128360,3,5.0,...,16974,1,21,3,0,1,0,0,1,1


In [13]:
td.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating              0
visitor_hist_adr_usd                 0
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
random_bool                          0
click_bool               

In [78]:
null_score = td[td["prop_location_score2"].isnull()]["prop_id"]

# get rows where prop_location_score2 is not null
not_null_score = td[td["prop_location_score2"].notnull()][["prop_id", "prop_location_score2"]]

# get mean of prop_location_score2 for each prop_id
mean_score = not_null_score.groupby("prop_id").mean()
mean_dict = mean_score.to_dict()["prop_location_score2"]

# fill null values with mean
td.loc[td["prop_location_score2"].isnull(), "prop_location_score2"] = td[td["prop_location_score2"].isnull()]["prop_id"].map(mean_dict)


In [81]:
td["prop_location_score2"].fillna(-1, inplace=True)
td["prop_review_score"].fillna(0, inplace=True)

In [82]:
td.isnull().sum()

srch_id                        0
date_time                      0
site_id                        0
visitor_location_country_id    0
visitor_hist_starrating        0
visitor_hist_adr_usd           0
prop_country_id                0
prop_id                        0
prop_starrating                0
prop_review_score              0
prop_brand_bool                0
prop_location_score1           0
prop_location_score2           0
prop_log_historical_price      0
position                       0
price_usd                      0
promotion_flag                 0
srch_destination_id            0
srch_length_of_stay            0
srch_booking_window            0
srch_adults_count              0
srch_children_count            0
srch_room_count                0
srch_saturday_night_bool       0
random_bool                    0
click_bool                     0
booking_bool                   0
dtype: int64

In [83]:
td.shape

(4958347, 27)

In [134]:
booked_hotels = td[td["booking_bool"] == 1]

booked_scores = booked_hotels[["prop_id", "prop_starrating", "prop_review_score", "prop_location_score1", "prop_location_score2"]]

In [135]:
# get how many times each hotel was booked
booked_count = booked_scores.groupby("prop_id").count()

# get mean of each score for each hotel
booked_mean = booked_scores.groupby("prop_id").mean()

# merge the datasets
booked = pd.merge(booked_count, booked_mean, on="prop_id")
booked.drop(["prop_starrating_x", "prop_review_score_x", "prop_location_score1_x"], axis=1, inplace=True)
booked.columns = ["booked_count", "prop_starrating", "prop_review_score", "prop_location_score1", "prop_location_score2"]
booked

,booked_count,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2
prop_id,,,,,
2,1,0.0,4.0,0.69,0.036100
3,2,3.0,3.5,0.69,0.011700
4,1,5.0,4.5,4.88,0.115500
11,4,4.0,4.0,4.65,0.124600
15,2,4.0,3.5,2.08,0.025200
...,...,...,...,...,...
140804,3,2.0,4.0,1.61,0.500300
140805,2,3.0,4.0,1.79,0.346100
140809,3,2.0,3.5,3.56,0.140067


In [136]:
# get the family hotels
family_hotels = booked_hotels[booked_hotels["srch_children_count"] > 0]
family_hotels = family_hotels[["prop_id"]].value_counts()
family_hotels = pd.DataFrame(family_hotels).reset_index().rename(columns={"count": "booked_family_count"})

# merge the datasets
booked = pd.merge(booked, family_hotels, on="prop_id", how="left")

In [137]:
booked

,prop_id,booked_count,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,booked_family_count
0,2,1,0.0,4.0,0.69,0.036100,NaN
1,3,2,3.0,3.5,0.69,0.011700,NaN
2,4,1,5.0,4.5,4.88,0.115500,1.0
3,11,4,4.0,4.0,4.65,0.124600,1.0
4,15,2,4.0,3.5,2.08,0.025200,1.0
...,...,...,...,...,...,...,...
43423,140804,3,2.0,4.0,1.61,0.500300,NaN
43424,140805,2,3.0,4.0,1.79,0.346100,1.0
43425,140809,3,2.0,3.5,3.56,0.140067,NaN
43426,140816,18,3.0,4.0,2.77,0.067733,7.0


In [138]:
# adult hotels
adult_hotels = booked_hotels[booked_hotels["srch_adults_count"] == 2]
adult_hotels = adult_hotels[adult_hotels["srch_children_count"] == 0]
adult_hotels = adult_hotels[["prop_id"]].value_counts()
adult_hotels = pd.DataFrame(adult_hotels).reset_index().rename(columns={"count": "booked_adult_count"})
adult_hotels

,prop_id,booked_adult_count
0,116942,72
1,22578,70
2,77089,66
3,137997,63
4,37818,57
...,...,...
29317,59505,1
29318,59504,1
29319,59498,1
29320,59493,1


In [139]:
# merge the datasets
booked = pd.merge(booked, adult_hotels, on="prop_id", how="left")

In [140]:
booked

,prop_id,booked_count,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,booked_family_count,booked_adult_count
0,2,1,0.0,4.0,0.69,0.036100,NaN,1.0
1,3,2,3.0,3.5,0.69,0.011700,NaN,1.0
2,4,1,5.0,4.5,4.88,0.115500,1.0,NaN
3,11,4,4.0,4.0,4.65,0.124600,1.0,1.0
4,15,2,4.0,3.5,2.08,0.025200,1.0,NaN
...,...,...,...,...,...,...,...,...
43423,140804,3,2.0,4.0,1.61,0.500300,NaN,NaN
43424,140805,2,3.0,4.0,1.79,0.346100,1.0,1.0
43425,140809,3,2.0,3.5,3.56,0.140067,NaN,3.0
43426,140816,18,3.0,4.0,2.77,0.067733,7.0,9.0


In [141]:
# get couples
couples = booked_hotels[booked_hotels["srch_adults_count"] == 2]
couples = couples[couples["srch_children_count"] == 0]
couples = couples[couples["srch_room_count"] == 1]

# get search ids of couples
couples_search_ids = couples["srch_id"].unique()

# get couples hotels
couples_hotels = booked_hotels[booked_hotels["srch_id"].isin(couples_search_ids)]
couples_hotels = couples_hotels[["prop_id"]].value_counts()
couples_hotels = pd.DataFrame(couples_hotels).reset_index().rename(columns={"count": "booked_couples_count"})

In [142]:
couples_hotels

,prop_id,booked_couples_count
0,116942,71
1,22578,67
2,77089,64
3,137997,60
4,37818,56
...,...,...
28825,59426,1
28826,59400,1
28827,59393,1
28828,59362,1


In [143]:
booked = pd.merge(booked, couples_hotels, on="prop_id", how="left")
booked

,prop_id,booked_count,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,booked_family_count,booked_adult_count,booked_couples_count
0,2,1,0.0,4.0,0.69,0.036100,NaN,1.0,1.0
1,3,2,3.0,3.5,0.69,0.011700,NaN,1.0,1.0
2,4,1,5.0,4.5,4.88,0.115500,1.0,NaN,NaN
3,11,4,4.0,4.0,4.65,0.124600,1.0,1.0,1.0
4,15,2,4.0,3.5,2.08,0.025200,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
43423,140804,3,2.0,4.0,1.61,0.500300,NaN,NaN,NaN
43424,140805,2,3.0,4.0,1.79,0.346100,1.0,1.0,1.0
43425,140809,3,2.0,3.5,3.56,0.140067,NaN,3.0,3.0
43426,140816,18,3.0,4.0,2.77,0.067733,7.0,9.0,9.0
